# Технології ETL з бібліотекою `scikit-learn`

## УВЕДЕННЯ В SCIKIT-LEARN

### Загальна інформація

Розроблено для побудови та аналізу класичних моделей машинного навчання [(Докладніше)](https://en.wikipedia.org/wiki/Scikit-learn)
- [Офіційний сайт](https://scikit-learn.org/stable/)
- [Сайт на GitHub](https://github.com/scikit-learn/scikit-learn)

Альтернативи:
- [Tensorflow](https://www.tensorflow.org)
- [Pytorch](https://pytorch.org)
- [Keras](https://keras.io) ...

Доступна через [API](https://scikit-learn.org/stable/modules/classes.html#) з мови Python (або С/С++)

### Інструментарій для роботи з данними

Має потужні інструменти для [Data Transformation](https://scikit-learn.org/stable/data_transforms.html#data-transforms)

Автоматизують наступні функції
- [очищення](https://scikit-learn.org/stable/modules/unsupervised_reduction.html)
- [вилучення](https://scikit-learn.org/stable/modules/feature_extraction.html)
- [збагачення](https://scikit-learn.org/stable/modules/impute.html)
- [предобробка](https://scikit-learn.org/stable/modules/preprocessing.html)

Для інтеграції цих (та інших) процесів передбачена [`конвереїзація`](https://scikit-learn.org/stable/modules/compose.html)


## ТЕОРЕТИЧНА ЧАСТИНА ТА ПРИКЛАДИ

### для проведення експеріментів та перевірки гіпотез в `scilearn` є модуль моделювання датасетів [sklearn.datasets](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets)

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np

In [ ]:
wine_bunch = datasets.load_wine(as_frame=True)

In [ ]:
df = wine_bunch['frame']

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.loc[df['target'] == 0,'target'] = 'west'
df.loc[df['target'] == 1,'target'] = 'east'
df.loc[df['target'] == 2,'target'] = 'nord'

In [ ]:
df['target'].unique()

In [ ]:
df.rename(columns={'target': 'region'}, inplace=True)
df.columns

In [ ]:
X, y = df.drop(columns=['alcohol']), df['alcohol']

In [ ]:
X.head(3)

### додамо синтетичний `шум' 

In [ ]:
from add_noise import add_feauture_err

In [ ]:
# деякі значення можуть бути пропущені
X = add_feauture_err(df=X, column_name='region', err_level=.05, err_filler=np.nan)
X = add_feauture_err(df=X, column_name='color_intensity', err_level=.10, err_filler=np.nan)

# або вочевидь невірні
X = add_feauture_err(df=X, column_name='magnesium', err_level=.05, err_filler=100000.0)

In [ ]:
X.describe()

In [ ]:
X.info()

### ПОШУК ТА ОБРОБКА ВИКИДІВ (Outliers/Anomaly Detection)

В продашені та на великих об'ємах використовується алгоритм __LOF__ ([_Local Outlier Factor_](https://scikit-learn.org/stable/auto_examples/neighbors/plot_lof_outlier_detection.html))

In [ ]:
mean_std = X.describe().loc[['mean', 'std']].T

In [ ]:
mean_std

In [ ]:
mean_std['std_minus_mean'] = mean_std['std'] - mean_std['mean']
mean_std

In [ ]:
# використаємо правило 2-х сігм для показчика `magnesium`
two_sigma = 2 * mean_std.loc['magnesium']['std']
two_sigma

In [ ]:
# замінити всі значення `magnesium` на медіану 
X.loc[df['magnesium'] > two_sigma] = np.median(X['magnesium'])

### заміщення пропусків

[Детальніше](https://scikit-learn.org/stable/modules/impute.html)

In [ ]:
X.describe().loc['count']

In [ ]:
X['color_intensity'].isna()

In [ ]:
# замінимо NaN на медіану
X['color_intensity'].fillna(value=X['color_intensity'].median(), inplace=True)

In [ ]:
# для строкового показчика - замінимо найближчим
X['region'].fillna(method='bfill', inplace=True)

In [ ]:
X.info()

### Стандартизація та масштабування числових показчиків

[Докладніше](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling)

In [ ]:
# відбір показчиків за їх типом
from sklearn.compose import make_column_selector
numeric_columns_selector = make_column_selector(dtype_include='float64')
X_numeric_columns = numeric_columns_selector(X)
X_numeric_columns

In [ ]:
X_numeric = X[X_numeric_columns]
X_numeric.info()

In [ ]:
# підключимо стандартизатор
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
_ = scaler.fit(X_numeric)

In [ ]:
(scaler.mean_, scaler.scale_, scaler.var_)

In [ ]:
X_numeric_scaled = pd.DataFrame(scaler.transform(X_numeric), columns=X_numeric_columns)

In [ ]:
X_numeric_scaled.describe()

### Перетворення категоріальних даних

[Докладніше про стратегії кодування](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features)

In [ ]:
categorical_selector = make_column_selector(dtype_include='O')
X_categorical_columns = categorical_selector(X)
X_categorical_columns

In [ ]:
X['region'].unique()

In [ ]:
X_categorical = X[X_categorical_columns]
X_categorical.describe()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [ ]:
_ = encoder.fit(X_categorical)

In [ ]:
X_categorical_encoded = encoder.transform(X_categorical)
X_categorical_encoded = pd.DataFrame(X_categorical_encoded, columns=encoder.categories_)

### Об'єднати числовий та категоріальний датасети

In [ ]:
(X_numeric_scaled.shape, X_categorical_encoded.shape)

In [ ]:
X = pd.concat([X_numeric_scaled, X_categorical_encoded], axis=1)

In [ ]:
X.head()

## ІНДИВІДУАЛЬНЕ ЗАВДАННЯ

In [1]:
import warnings
warnings.filterwarnings('ignore')

<div style='background-color:lightgreen'>
<H3 style='color:red'>Виконати процедури очищення, збагачення та нормалізації даних рієлторської компанії в файлі `aprt_properties.csv`</H3>
<p>Н<b>а місці `...` написати свій код<b></p>
</div>


In [1]:
# імпортувати бібліотеку Pandas, NumPy 
import numpy as np
import pandas as pd

### I. Вилучення сирих даних

__Примітка__: використати функцію `read_csv` з параметром `na_values=['None']` щоб згенерувати правильні типи даних

In [2]:
df = pd.read_csv('aprt_properties.csv', na_values=None)


### II. Очищення (_опціонально_)

__Примітка__: на цьому етапі слід
1. Визначитись які показчики є категоріальними або числовими
2. При необхідності зробити перетворення типів (функція `astype`)
3. Визначити чи є в датасеті викиди.
4. При наявності викидів зробити їх корегування (`mean` або `median`)

### III. Заповнення пропусків для чисельних показчиків

__Примітка__: на цьому етапі слід

1. Виділити датасет з чисельними показчиками використовуючи `make_column_selector`
2. Заповнити пусті (`NaN`) значення медіанними значеннями відповідної колонки (`fillna`)

In [59]:
from sklearn.compose import make_column_selector
from add_noise import add_feauture_err
numeric_columns_selector = make_column_selector(dtype_include='float64')

X_numeric_columns = numeric_columns_selector(df)
X_numeric_columns
X = add_feauture_err(df=df, column_name='Price', err_level=.05, err_filler=np.nan)
X.describe()
X.isna()
X.fillna(value=X.median, inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Price         728 non-null    object
 1   rooms         728 non-null    object
 2   price_per_m2  728 non-null    object
 3   level         728 non-null    object
 4   levels        728 non-null    object
 5   year          728 non-null    object
 6   area_total    728 non-null    object
 7   area_living   728 non-null    object
 8   area_kitchen  728 non-null    object
 9   street        728 non-null    object
 10  publish_date  728 non-null    object
dtypes: object(11)
memory usage: 62.7+ KB


### IV. Масштабування чисельних показчиків

__Примітка__: на цьому етапі слід побудувати змаштабований датасет `df_num_scaled` з чисельних показчиків використавші модуль __`StandardScaler`__ та його функцію `fit_transform`

In [74]:
from sklearn.preprocessing import StandardScaler
# df = pd.read_csv('aprt_properties.csv', na_values=None)
X_numeric_columns = numeric_columns_selector(X)
X_numeric_columns
X_numeric = X[X_numeric_columns]
X_numeric.info()
scaler = StandardScaler()
_ = scaler.fit(X_numeric)
(scaler.mean_, scaler.scale_, scaler.var_)
df_num_scaled = pd.DataFrame(scaler.fit_transform(X_numeric), columns=X_numeric_columns)
df_num_scaled

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Price   728 non-null    float64
dtypes: float64(1)
memory usage: 5.8 KB


,Price
0,-0.089169
1,-0.141065
2,-0.037274
3,-0.250832
4,-0.175168
...,...
723,-0.222630
724,-0.164789
725,-0.044687
726,-0.253372


### V. Заповнення пропусків для категоріальних показчиків

__Примітка__: на цьому етапі слід
1. Виділити датасет з категоріальними показчиками використовуючи `make_column_selector` з параметром `dtype_include='O'`
2. Видилити колонку `publish_date` бо вона в цьому датасеті чисто інформативна
3. Заповнити пусті (NaN) значення найближчими (`method='ffill'` в функції `fillna`)

In [102]:
# df = pd.read_csv('aprt_properties.csv', na_values=None)
categorical_selector = make_column_selector(dtype_include='O')
X_categorical_columns = categorical_selector(X)
X_categorical_columns
X = add_feauture_err(df=df, column_name='publish_date', err_level=.05, err_filler=np.nan)
X.describe()
X.isna()
X.fillna(method='ffill', inplace=True)
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Price         728 non-null    float64
 1   rooms         728 non-null    object 
 2   price_per_m2  728 non-null    object 
 3   level         728 non-null    object 
 4   levels        728 non-null    object 
 5   year          728 non-null    object 
 6   area_total    728 non-null    object 
 7   area_living   728 non-null    object 
 8   area_kitchen  728 non-null    object 
 9   street        728 non-null    object 
 10  publish_date  728 non-null    object 
dtypes: float64(1), object(10)
memory usage: 62.7+ KB


### VI. Кодування категориальних показчиків

__Примітка__: на цьому етапі слід побудувати датасет `df_cat_encoded` використавши функцію  `fit_transform` модуля __`OneHotEncoder`__


In [6]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [111]:
categorical_selector = make_column_selector(dtype_include='O')
X_categorical_columns = categorical_selector(X)
X_categorical_columns
X_categorical = X[X_categorical_columns]
# X_categorical.info()
_ = encoder.fit(X_categorical)
df_cat_encoded = encoder.fit_transform(X_categorical)

df_cat_encoded = pd.DataFrame(df_cat_encoded) # Якщо застосувати columns=encoder.cartegories_ - видає помилку, так як при використанні fir_transfrom генерує понад 1500 стовбців, хоча все виконано по аналогії з теоретичними прикладами
df_cat_encoded


,0,1,2,3,4,5,6,7,8,9,...,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
724,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
725,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
726,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### VII. Формування результатного датасету

__Примітка__: на цьому етапі слід об'єднати змаштабований чисельний (`df_num_scaled`) та кодований категоріальний (`df_cat_encoded`) датасети в один результатний `df_result` використавши функцію `concat`

In [112]:
X = pd.concat([df_num_scaled, df_cat_encoded], axis=1)
X.head()

,Price,0,1,2,3,4,5,6,7,8,...,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510
0,-0.089169,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.141065,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.037274,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.250832,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.175168,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
